In [11]:
import pandas as pd
from transformers import pipeline, set_seed
from sentence_transformers import SentenceTransformer
import requests
from bs4 import BeautifulSoup

In [2]:
encoder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

/opt/anaconda/lib/python3.11/site-packages/huggingface_hub-0.23.0-py3.8.egg/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.


In [4]:
picture_info = pd.read_csv('../data/best_picture_2000.csv')

In [7]:
picture_info['link'][0]

'https://en.wikipedia.org/wiki/Gladiator_(2000_film)'

In [9]:
URL = picture_info['link'][0]

response = requests.get(URL)
response.status_code

200

In [10]:
response.text

'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-0 vector-feature-appearance-disabled vector-feature-appearance-pinned-clientpref-0 vector-feature-night-mode-disabled skin-theme-clientpref-day vector-toc-available" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8">\n<title>Gladiator (2000 film) - Wikipedia</title>\n<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled ve

In [12]:
soup = BeautifulSoup(response.text)

In [15]:
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-0 vector-feature-appearance-disabled vector-feature-appearance-pinned-clientpref-0 vector-feature-night-mode-disabled skin-theme-clientpref-day vector-toc-available" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Gladiator (2000 film) - Wikipedia
  </title>
  <script>
   (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinne

In [18]:
# Find all paragraphs within the main content area
paragraphs = soup.find(id='mw-content-text').find_all('p')

# Extract plain text from paragraphs using get_text() method
text = '\n'.join([p.get_text() for p in paragraphs])


In [20]:
print(text)



Gladiator is a 2000 historical epic film directed by Ridley Scott and written by David Franzoni, John Logan, and William Nicholson. It stars Russell Crowe, Joaquin Phoenix, Connie Nielsen, Tomas Arana, Ralf Möller, Oliver Reed (in his final role), Djimon Hounsou, Derek Jacobi, John Shrapnel, Richard Harris, and Tommy Flanagan.[4][5][6]

Crowe portrays Roman general Maximus Decimus Meridius, who is betrayed when Commodus, the ambitious son of Emperor Marcus Aurelius, murders his father and seizes the throne. Reduced to slavery, Maximus becomes a gladiator and rises through the ranks of the arena, determined to avenge the murders of his family and the emperor.

The screenplay, initially written by Franzoni, was inspired by the 1958 Daniel P. Mannix novel Those About to Die. The script was acquired by DreamWorks Pictures, and Scott signed on to direct the film. Principal photography began in January 1999 and wrapped in May of that year. Production was complicated by the script being rew

In [22]:
picture_info.head()

,title,link,year
0,Gladiator,https://en.wikipedia.org/wiki/Gladiator_(2000_...,2000
1,A Beautiful Mind,https://en.wikipedia.org/wiki/A_Beautiful_Mind...,2001
2,Chicago,https://en.wikipedia.org/wiki/Chicago_(2002_film),2002
3,The Lord of the Rings: The Return of the King,https://en.wikipedia.org/wiki/The_Lord_of_the_...,2003
4,Million Dollar Baby,https://en.wikipedia.org/wiki/Million_Dollar_Baby,2004


In [21]:
urls = picture_info['link']

In [25]:
urls

0     https://en.wikipedia.org/wiki/Gladiator_(2000_...
1     https://en.wikipedia.org/wiki/A_Beautiful_Mind...
2     https://en.wikipedia.org/wiki/Chicago_(2002_film)
3     https://en.wikipedia.org/wiki/The_Lord_of_the_...
4     https://en.wikipedia.org/wiki/Million_Dollar_Baby
5       https://en.wikipedia.org/wiki/Crash_(2004_film)
6            https://en.wikipedia.org/wiki/The_Departed
7     https://en.wikipedia.org/wiki/No_Country_for_O...
8     https://en.wikipedia.org/wiki/Slumdog_Millionaire
9         https://en.wikipedia.org/wiki/The_Hurt_Locker
10    https://en.wikipedia.org/wiki/The_King%27s_Speech
11      https://en.wikipedia.org/wiki/The_Artist_(film)
12       https://en.wikipedia.org/wiki/Argo_(2012_film)
13    https://en.wikipedia.org/wiki/12_Years_a_Slave...
14         https://en.wikipedia.org/wiki/Birdman_(film)
15       https://en.wikipedia.org/wiki/Spotlight_(film)
16    https://en.wikipedia.org/wiki/Moonlight_(2016_...
17     https://en.wikipedia.org/wiki/The_Shape_o

In [26]:
all_text = []
for url in urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.text)
    # Find all paragraphs within the main content area
    paragraphs = soup.find(id='mw-content-text').find_all('p')
    # Extract plain text from paragraphs using get_text() method
    article_text = '\n'.join([p.get_text() for p in paragraphs])
    all_text.append(article_text)

In [27]:
all_text

['\n\nGladiator is a 2000 historical epic film directed by Ridley Scott and written by David Franzoni, John Logan, and William Nicholson. It stars Russell Crowe, Joaquin Phoenix, Connie Nielsen, Tomas Arana, Ralf Möller, Oliver Reed (in his final role), Djimon Hounsou, Derek Jacobi, John Shrapnel, Richard Harris, and Tommy Flanagan.[4][5][6]\n\nCrowe portrays Roman general Maximus Decimus Meridius, who is betrayed when Commodus, the ambitious son of Emperor Marcus Aurelius, murders his father and seizes the throne. Reduced to slavery, Maximus becomes a gladiator and rises through the ranks of the arena, determined to avenge the murders of his family and the emperor.\n\nThe screenplay, initially written by Franzoni, was inspired by the 1958 Daniel P. Mannix novel Those About to Die. The script was acquired by DreamWorks Pictures, and Scott signed on to direct the film. Principal photography began in January 1999 and wrapped in May of that year. Production was complicated by the script b

In [28]:
embeddings = encoder.encode(all_text)

In [29]:
decoder = pipeline('text-generation', model='distilgpt2')

/opt/anaconda/lib/python3.11/site-packages/huggingface_hub-0.23.0-py3.8.egg/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.


config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [30]:
qa = pipeline(task='question-answering')

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
/opt/anaconda/lib/python3.11/site-packages/huggingface_hub-0.23.0-py3.8.egg/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [31]:
questions = pd.read_csv('../data/QAs.csv')

In [35]:
questions_ = questions['question']